# Consultas

In [ ]:
#r "nuget: Microsoft.DotNet.Interactive.SqlServer, *-*"

In [ ]:
foreach (var line in System.IO.File.ReadLines(".env"))
{
    var parts = line.Split('=');
    if (parts.Length == 2)
    {
        var key = parts[0].Trim();
        var value = parts[1].Trim();
        Environment.SetEnvironmentVariable(key, value);
    }
}

string connectionString = "Server=" + Environment.GetEnvironmentVariable("Server") + ";Database=" + Environment.GetEnvironmentVariable("Database") + ";User Id=" + Environment.GetEnvironmentVariable("User") + ";Password=" + Environment.GetEnvironmentVariable("Password") + ";Encrypt=True;TrustServerCertificate=True;Connection Timeout=30;";


In [ ]:
#!connect mssql --kernel-name Empleados @csharp:connectionString

In [ ]:
DECLARE @id_emp INT = 15678;
DECLARE @id_dept CHAR(4) = 'd008';

-- a) Devolver los datos (id, fecha de nacimiento, nombre y apellido concatenados, género y fecha de alta) del empleado asignado si alguna vez trabajó en el departamento asignado.
SELECT id_emp, fecha_nacimiento, CONCAT(nombre, ' ', apellido) AS nombre_completo, genero, fecha_alta
FROM empleados 
WHERE id_emp = @id_emp
AND EXISTS ( SELECT 1 FROM dept_emp WHERE id_dept = @id_dept AND id_emp = @id_emp);

-- b) Devolver los datos (id, fecha de nacimiento, nombre y apellido concatenados, género y fecha de alta) del empleado asignado si no trabaja actualmente en el departamento asignado.
SELECT id_emp, fecha_nacimiento, CONCAT(nombre, ' ', apellido) AS nombre_completo, genero, fecha_alta
FROM empleados
WHERE id_emp = @id_emp
AND NOT EXISTS ( SELECT 1 FROM dept_emp WHERE id_dept = @id_dept AND id_emp = @id_emp AND fecha_hasta = '99990101');

-- c) Devolver los datos (id, nombre y apellido concatenados) del responsable actual del departamento asignado.
SELECT id_emp, CONCAT(nombre, ' ', apellido) AS nombre_completo
FROM empleados
WHERE id_emp = (SELECT id_emp FROM dept_respo WHERE id_dept = @id_dept AND fecha_hasta = '99990101');

-- d) Devolver los datos del empleado asignado, incluyendo el nombre del departamento en el que trabaja actualmente (realizar la consulta considerando la posibilidad de que el empleado ya no trabaje en la compañía, y en ese caso se debería mostrar el último departamento al que estuvo asignado).
-- Con Subconsulta
SELECT E.id_emp, fecha_nacimiento, CONCAT(nombre, ' ', apellido) AS nombrecompleto, genero, fecha_alta, D.nombre_dept
FROM empleados AS E JOIN dept_emp AS DE ON E.id_emp = DE.id_emp
JOIN departamentos AS D ON DE.id_dept = D.id_dept
WHERE E.id_emp = @id_emp AND
DE.fecha_hasta = (SELECT MAX(fecha_hasta) FROM dept_emp AS DEI WHERE DEI.id_emp = E.id_emp);
-- Con Order y Top
SELECT TOP 1 E.id_emp, fecha_nacimiento, CONCAT(nombre, ' ', apellido) AS nombrecompleto, genero, fecha_alta, D.nombre_dept
FROM empleados AS E JOIN dept_emp AS DE ON E.id_emp = DE.id_emp
JOIN departamentos AS D ON DE.id_dept = D.id_dept
WHERE E.id_emp = @id_emp 
ORDER BY DE.fecha_hasta DESC;

-- e) Agregar a la consulta anterior, el apellido del responsable del departamento.
SELECT E.id_emp, E.fecha_nacimiento, CONCAT(E.nombre, ' ', E.apellido) AS nombrecompleto, E.genero, E.fecha_nacimiento, D.nombre_dept, J.apellido
FROM empleados AS E JOIN dept_emp AS DE ON E.id_emp = DE.id_emp
JOIN departamentos AS D ON DE.id_dept = D.id_dept
JOIN dept_respo AS DR ON DR.id_dept = D.id_dept
JOIN empleados AS J ON J.id_emp = DR.id_emp
WHERE E.id_emp = @id_emp
AND DE.fecha_hasta = (SELECT MAX(fecha_hasta) FROM dept_emp AS DEI WHERE DEI.id_emp = E.id_emp)
AND DR.fecha_hasta = '99990101'; 

-- f) Considerando el menor y el mayor sueldo, ¿qué porcentaje de aumento recibió el empleado asignado? El formato de la respuesta deberá contar con dos decimales y el símbolo de porcentaje, por ejemplo 20.50%
SELECT CONCAT(CAST(MAX(sueldo)*100.00/MIN(sueldo)-100.00 AS DECIMAL(19,2)),'%') as Porcentaje
FROM sueldos
WHERE id_emp = @id_emp;

-- g) Obtener la lista de empleados (emp_id, fecha_nacimiento, nombre, apellido y género) que trabajan actualmente en el departamento asignado, cuyo sueldo actual es mayor a 120.000. El puesto que ocupa debe ser Ingeniero (el nombre del puesto debe contener “Engineer”) o tener categoría Senior (el nombre del puesto contiene “Senior”).
SELECT E.id_emp, fecha_nacimiento, E.nombre, E.apellido, E.genero
FROM dept_emp AS DE JOIN empleados AS E ON DE.id_emp=E.id_emp
JOIN puestos AS P ON P.id_emp=E.id_emp
JOIN puesto_descr AS PD ON P.id_puesto=PD.id_puesto
JOIN sueldos AS S ON E.id_emp=S.id_emp
WHERE DE.id_dept = @id_dept
AND DE.fecha_hasta = '99990101'
AND P.fecha_hasta = '99990101'
AND S.fecha_hasta = '99990101'
AND sueldo > 120000
AND (puesto LIKE '%Engineer%' OR puesto LIKE '%Senior%');

-- h) Agregar un nuevo departamento, con el código d010 y el nombre “Inteligencia Artificial”.
INSERT INTO departamentos (id_dept, nombre_dept) VALUES ('d010', 'Inteligencia Artificial');

-- i) Con vigencia desde el 01/01/2023, asigne los empleados obtenidos en la consulta g al nuevo departamento d010 y otórgueles un 15% de aumento de sueldo. Realice todos los comandos DML necesarios para que estos cambios queden reflejados correctamente en la base de datos.
BEGIN TRANSACTION;
UPDATE dept_emp SET fecha_hasta = '20230101'
WHERE fecha_hasta = '99990101' AND id_dept = @id_dept
AND id_emp IN (
SELECT E.id_emp
FROM dept_emp AS DE JOIN empleados AS E ON DE.id_emp=E.id_emp
JOIN puestos AS P ON P.id_emp=E.id_emp
JOIN puesto_descr AS PD ON P.id_puesto=PD.id_puesto
JOIN sueldos AS S ON E.id_emp=S.id_emp
WHERE DE.id_dept = @id_dept
AND DE.fecha_hasta = '99990101'
AND P.fecha_hasta = '99990101'
AND S.fecha_hasta = '99990101'
AND sueldo > 120000
AND (puesto LIKE '%Engineer%' OR puesto LIKE '%Senior%'));

INSERT INTO dept_emp (id_emp, id_dept, fecha_desde, fecha_hasta)
SELECT E.id_emp, 'd010', '20230101', '99990101'
FROM dept_emp AS DE JOIN empleados AS E ON DE.id_emp=E.id_emp
JOIN puestos AS P ON P.id_emp=E.id_emp
JOIN puesto_descr AS PD ON P.id_puesto=PD.id_puesto
JOIN sueldos AS S ON E.id_emp=S.id_emp
WHERE DE.id_dept = @id_dept
AND DE.fecha_hasta = '20230101'
AND P.fecha_hasta = '99990101'
AND S.fecha_hasta = '99990101'
AND sueldo > 120000
AND (puesto LIKE '%Engineer%' OR puesto LIKE '%Senior%');

UPDATE sueldos SET fecha_hasta = '20230101'
WHERE fecha_hasta = '99990101' 
AND id_emp IN (
SELECT E.id_emp
FROM empleados AS E JOIN dept_emp AS DE ON E.id_emp=DE.id_emp
WHERE DE.id_dept='d010'
);

INSERT INTO sueldos (id_emp, sueldo, fecha_desde, fecha_hasta)
SELECT E.id_emp, sueldo * 1.15 as aumento, '20230101', '99990101'
FROM empleados AS E JOIN dept_emp AS DE ON E.id_emp=DE.id_emp
JOIN sueldos AS S ON E.id_emp = S.id_emp
WHERE DE.id_dept='d010' AND S.fecha_hasta = '20230101';
COMMIT;

In [ ]:
-- j) Liste la cantidad de empleados de empleados y sueldo promedio de cada departamento, considerado solamente a los empleados que trabajan actualmente.
SELECT D.id_dept, D.nombre_dept, COUNT(E.id_emp) AS CantEmpl, AVG(CONVERT(DECIMAL(19,4),S.sueldo)) AS SueldoProm
FROM departamentos AS D JOIN dept_emp AS DE ON D.id_dept = DE.id_dept
JOIN empleados AS E ON DE.id_emp = E.id_emp
JOIN sueldos AS S ON E.id_emp = S.id_emp
WHERE DE.fecha_hasta = '99990101'
AND S.fecha_hasta = '99990101'
GROUP BY D.id_dept, D.nombre_dept
ORDER BY D.id_dept, D.nombre_dept;